In [1]:
import time

import torch
import torch.nn as nn
import torch.nn.functional as F

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
device

device(type='cpu')

In [3]:
import src.CayleyNet as CayleyNet
import src.CORA as CORA
import src.utils as utils

In [4]:
# Reload when files are changed
import importlib
importlib.reload(CayleyNet)
importlib.reload(CORA)
importlib.reload(utils)

<module 'src.utils' from 'c:\\Users\\Usuario\\Documents\\Documents\\MVA\\GDA\\Projet\\CayleyNets\\src\\utils.py'>

In [5]:
# Import dataset
cora = CORA.CORA()

Unpacking archive...
Parsing files...
Done.


In [6]:
# Print properties of the dataset
dataset = cora
print(dataset.description)
print(f'Number of nodes: {dataset.n}')
print(f'Number of edges: {dataset.n_edges}')
print(f'Number of features per node: {dataset.n_features}')

CORA dataset
Number of nodes: 2708
Number of edges: 10556
Number of features per node: 1433


In [7]:
# Extended split of the CORA dataset
train_mask, test_mask, val_mask = utils.split_train_test_val(cora.n, 500, 500) 

In [8]:
# Features and labels as tensors
features = torch.Tensor(1.0*dataset.features).to(device)
labels = torch.Tensor(dataset.labels).long().to(device)
edge_index = dataset.get_edge_index()

# Masks as tensors
train_mask = torch.Tensor(train_mask).bool().to(device)
test_mask = torch.Tensor(test_mask).bool().to(device)
val_mask = torch.Tensor(val_mask).bool().to(device)

In [9]:
# Hyperparameters (Section 4.5)
in_feats = dataset.n_features
n_classes = dataset.n_classes
n_hidden = 16
n_layers = 1 # number of hidden+output layers
r = 5 # Cayley polynomial order 
# obs: we weren't able to identify the value of this last parameter in the paper

In [10]:
model = CayleyNet.CayleyNet(in_feats, n_classes, n_hidden, n_layers, r=5, p_dropout=0.5, normalization = 'sym', sparse=True, seed=0)

c:\Users\Usuario\anaconda3\envs\cayleynets\lib\site-packages\torch\nn\modules\module.py:1133: UserWarning: Complex modules are a new feature under active development whose design may change, and some modules might not work as expected when using complex tensors as parameters or buffers. Please file an issue at https://github.com/pytorch/pytorch/issues/new?template=bug-report.yml if a complex module does not work as expected.
  warnings.warn(


In [11]:
print(model)
print(f'Number of parameters: {sum(p.numel() for p in model.parameters())}')
print(f'Number of parameters: {sum(p.numel() for p in model.layers[0].parameters())}')
print(f'Number of parameters: {sum(p.numel() for p in model.layers[1].parameters())}')

CayleyNet(
  (layers): ModuleList(
    (0): CayleyConv(1433, 16, r=5, normalization=sym)
    (1): CayleyConv(16, 7, r=5, normalization=sym)
  )
)
Number of parameters: 138242
Number of parameters: 137569
Number of parameters: 673


In [12]:
# Optimizer
lr = 5e-3
weight_decay = 5e-4
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [13]:
# Train the model
batch_size = 32
epochs = 100

# save loss values for plotting
loss_values = []
val_score = []
train_score = []

verbose = True
for e in range(epochs):
    start = time.time()

    # Compute output
    logp = model(features, edge_index)

    # Compute loss
    loss = F.nll_loss(logp[train_mask], labels[train_mask])
    loss_values.append(loss.item())

    # Perform backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Evaluation
    score = utils.eval_pytorch_geometric_model(model, edge_index, features, labels, train_mask)
    train_score.append(score)

    score = utils.eval_pytorch_geometric_model(model, edge_index, features, labels, val_mask)
    val_score.append(score)

    # Print loss
    end = time.time()
    if verbose:
        print("Epoch {:02d} | Loss {:.3f} | Accuracy (validation) {:.3f} | Elapsed time: {:.2f}s".format(e, loss.item(), score, end - start))

torch.Size([2708, 1433])
torch.Size([2708, 1433])
torch.Size([2708, 1433])
torch.Size([2708, 1433])
torch.Size([2708, 16])
torch.Size([2708, 16])
torch.Size([2708, 16])
torch.Size([2708, 16])


RuntimeError: unsupported operation: more than one element of the written-to tensor refers to a single memory location. Please clone() the tensor before performing the operation.

In [ ]:
test_score = utils.eval_pytorch_geometric_model(model, edge_index, features, labels, test_mask)
print("Test accuracy {:.3f}".format(test_score))

In [ ]:
utils.plot_accuracy(train_score, val_score)

In [ ]:
utils.plot_loss(loss_values)